In [1]:
# Librerias necesarias para modelo de regresión logística
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import os
import json
import warnings
# from sklearn.exceptions import ConvergenceWarning


# # Suprimir advertencias de convergencia
# warnings.simplefilter("ignore", ConvergenceWarning)
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd

# Ruta base de los archivos
BASE_PATH = "../data/processed"

# Archivos de entrenamiento
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv",
    "X_train_con_outliers_norm.csv",
    "X_train_sin_outliers_norm.csv",
    "X_train_con_outliers_scal.csv",
    "X_train_sin_outliers_scal.csv"
]

# Archivos de prueba
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv",
    "X_test_con_outliers_norm.csv",
    "X_test_sin_outliers_norm.csv",
    "X_test_con_outliers_scal.csv",
    "X_test_sin_outliers_scal.csv"
]

# Cargar los datasets de entrenamiento
TRAIN_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]

# Cargar los datasets de prueba
TEST_DATASETS = [pd.read_csv(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]

# Cargar las etiquetas de entrenamiento y prueba
y_train = pd.read_csv(os.path.join(BASE_PATH, "y_train.csv")).values.ravel()
y_test = pd.read_csv(os.path.join(BASE_PATH, "y_test.csv")).values.ravel()

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Lista para almacenar los resultados
results = []

# Iterar sobre los datasets de entrenamiento
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)  # Imprimir el índice (0, 1, 2, ...)

    # Inicializar el modelo de regresión lineal
    model = LinearRegression()
    
    # Entrenamos el modelo con los datos de entrenamiento
    model.fit(dataset, y_train)
    
    # Realizamos las predicciones sobre el conjunto de entrenamiento
    y_pred_train = model.predict(dataset)
    
    # Realizamos las predicciones sobre el conjunto de prueba
    y_pred_test = model.predict(TEST_DATASETS[index])

    # Calcular R² para los datos de entrenamiento y prueba
    results.append(
        {
            "train": r2_score(y_train, y_pred_train),  # R² para el entrenamiento
            "test": r2_score(y_test, y_pred_test)  # R² para la prueba
        }
    )

# Mostrar los resultados
#print(results)

for result in results:
    print(f"Train: {result['train']} | Test: {result['test']}")

0
1
2
3
4
5
Train: 0.999693662967596 | Test: 0.9987413402763947
Train: 0.5418679444079675 | Test: 0.4797314823776383
Train: 0.999693662967596 | Test: 0.9987413402763814
Train: 0.5418679444079675 | Test: 0.47973148237674634
Train: 0.999693662967596 | Test: 0.9987413402763814
Train: 0.5418679444079676 | Test: 0.47973148237674457


In [4]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

# Lista para almacenar los resultados
results = []

# Iterar sobre los datasets de entrenamiento
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)  # Imprimir el índice (0, 1, 2, ...)

    # Inicializar el modelo de regresión Lasso
    model = Lasso()

    # Entrenamos el modelo con los datos de entrenamiento
    model.fit(dataset, y_train)

    # Realizamos las predicciones sobre el conjunto de entrenamiento
    y_pred_train = model.predict(dataset)

    # Realizamos las predicciones sobre el conjunto de prueba
    y_pred_test = model.predict(TEST_DATASETS[index])

    # Calcular R² para los datos de entrenamiento y prueba
    results.append(
        {
            "train": r2_score(y_train, y_pred_train),  # R² para el entrenamiento
            "test": r2_score(y_test, y_pred_test)  # R² para la prueba
        }
    )

# Imprimir los resultados sin redondear
for result in results:
    print(f"Train: {result['train']} | Test: {result['test']}")

0
1
2
3
4
5
Train: 0.9995363037266238 | Test: 0.9982356855988306
Train: 0.5409204276834187 | Test: 0.48238564390482097
Train: 0.9995371273791716 | Test: 0.9982627338537655
Train: 0.5409183267006301 | Test: 0.4824653773074835
Train: 0.9995518210703317 | Test: 0.9985892726254676
Train: 0.5409000830631617 | Test: 0.48313510536096627


In [5]:
from tqdm import tqdm  # Importar tqdm para la barra de progreso

# Definir los rangos de hiperparámetros que se probarán
hyperparams = {
    "alpha": np.logspace(-6, 6, 3),  # Rango de valores de alpha
    "fit_intercept": [True, False],  # Ajuste del término de intersección
    "max_iter": [1000, 5000],  # Número máximo de iteraciones
    "tol": [1e-4, 1e-3, 1e-2]  # Tolerancia de convergencia
}

# Inicializar el modelo Lasso sin entrenar
model = Lasso(random_state=95)

# Usamos GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(model, hyperparams, scoring="r2", cv=5)  # Usamos validación cruzada de 5 pliegues

# Inicializamos los resultados
results = []

# Agregamos la barra de progreso con tqdm para iterar por cada dataset
for index, dataset in tqdm(enumerate(TRAIN_DATASETS), total=len(TRAIN_DATASETS), desc="Procesando datasets"):
    # Ajustamos el modelo con los datos de entrenamiento
    grid_search.fit(dataset, y_train)

    # Mejor conjunto de hiperparámetros encontrados
    best_params = grid_search.best_params_
    print(f"Mejores hiperparámetros para el dataset {index + 1}: {best_params}")

    # Mejor modelo con los parámetros optimizados
    best_model = grid_search.best_estimator_

    # Hacemos las predicciones con el mejor modelo en los datos de entrenamiento y test
    y_pred_train = best_model.predict(dataset)
    y_pred_test = best_model.predict(TEST_DATASETS[index])

    # Evaluar el rendimiento del modelo en los conjuntos de datos
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)

    # Almacenar los resultados para cada dataset
    results.append({
        "train": train_r2,
        "test": test_r2
    })

# Mostrar todos los resultados obtenidos
print(results)

Procesando datasets:  17%|█▋        | 1/6 [00:17<01:27, 17.52s/it]

Mejores hiperparámetros para el dataset 1: {'alpha': np.float64(1.0), 'fit_intercept': True, 'max_iter': 5000, 'tol': 0.0001}


Procesando datasets:  33%|███▎      | 2/6 [00:56<01:59, 29.85s/it]

Mejores hiperparámetros para el dataset 2: {'alpha': np.float64(1.0), 'fit_intercept': True, 'max_iter': 1000, 'tol': 0.0001}


Procesando datasets:  50%|█████     | 3/6 [01:14<01:14, 24.81s/it]

...

Mejores hiperparámetros para el dataset 4: {'alpha': np.float64(1.0), 'fit_intercept': True, 'max_iter': 1000, 'tol': 0.0001}


Procesando datasets:  83%|████████▎ | 5/6 [01:49<00:19, 19.18s/it]

Mejores hiperparámetros para el dataset 5: {'alpha': np.float64(1.0), 'fit_intercept': False, 'max_iter': 5000, 'tol': 0.0001}


Procesando datasets: 100%|██████████| 6/6 [02:17<00:00, 22.88s/it]

Mejores hiperparámetros para el dataset 6: {'alpha': np.float64(1.0), 'fit_intercept': True, 'max_iter': 1000, 'tol': 0.0001}
[{'train': 0.9996664754615998, 'test': 0.9987330024930475}, {'train': 0.5409204276834187, 'test': 0.48238564390482097}, {'train': 0.9996688287151908, 'test': 0.9987984578407791}, {'train': 0.5409183267006301, 'test': 0.4824653773074835}, {'train': 0.9996016757808334, 'test': 0.9989880885079095}, {'train': 0.5409000830631617, 'test': 0.48313510536096627}]


In [6]:

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV


# Entrenamos el modelo con los nuevos parámetros encontrados por GridSearchCV
best_dataset = np.argmax([result["train"] + result["test"] for result in results])  # Elegimos el dataset con la mayor suma de R2 en train y test

print(f"El mejor dataset es el {best_dataset + 1} (índice {best_dataset})")

# Ahora optimizamos el modelo con el mejor dataset usando GridSearchCV

# Inicializamos el modelo Lasso
best_model = Lasso(random_state=95)

# Creamos la búsqueda de hiperparámetros con los datos del mejor dataset
grid = GridSearchCV(best_model, hyperparams, scoring="r2", cv=5)  # Usamos validación cruzada

# Ajustamos el modelo en el mejor dataset
grid.fit(TRAIN_DATASETS[best_dataset], y_train)

# Mostramos los mejores hiperparámetros encontrados para este dataset
print(f"Mejores parámetros después de la optimización para el dataset {best_dataset + 1}: {grid.best_params_}")

# El mejor modelo con los nuevos parámetros optimizados
best_optimized_model = grid.best_estimator_

# Realizamos las predicciones con el modelo optimizado
y_pred_train_optimized = best_optimized_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test_optimized = best_optimized_model.predict(TEST_DATASETS[best_dataset])

# Evaluamos el rendimiento del modelo optimizado
train_r2_optimized = r2_score(y_train, y_pred_train_optimized)
test_r2_optimized = r2_score(y_test, y_pred_test_optimized)

# Imprimimos el desempeño del modelo optimizado
print(f"R2 optimizado en train: {train_r2_optimized:.5f}")
print(f"R2 optimizado en test: {test_r2_optimized:.5f}")

El mejor dataset es el 5 (índice 4)
Mejores parámetros después de la optimización para el dataset 5: {'alpha': np.float64(1.0), 'fit_intercept': False, 'max_iter': 5000, 'tol': 0.0001}
R2 optimizado en train: 0.99960
R2 optimizado en test: 0.99899


In [7]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score



# Definir los rangos de hiperparámetros para el modelo Ridge
param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100],  # Valores de alpha
    "max_iter": [100, 300, 1000, 2000, 4000, 8000, 12000],  # Número máximo de iteraciones
    "tol": np.linspace(0.0001, 0.01, 100),  # Tolerancia de convergencia
}

# Inicializar el modelo Ridge sin entrenar
ridge_model = Ridge(random_state=95)

# Usamos GridSearchCV para encontrar los mejores hiperparámetros
grid_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)

# Inicializamos los resultados
results_ridge = []

# Agregamos la barra de progreso con tqdm para iterar por cada dataset
for index, dataset in tqdm(enumerate(TRAIN_DATASETS), total=len(TRAIN_DATASETS), desc="Procesando datasets Ridge"):
    # Ajustamos el modelo con los datos de entrenamiento
    grid_ridge.fit(dataset, y_train)

    # Mejor conjunto de hiperparámetros encontrados
    best_params = grid_ridge.best_params_
    print(f"Mejores hiperparámetros para el dataset {index + 1}: {best_params}")

    # Mejor modelo con los parámetros optimizados
    best_ridge_model = grid_ridge.best_estimator_

    # Hacemos las predicciones con el mejor modelo en los datos de entrenamiento y test
    y_pred_train = best_ridge_model.predict(dataset)
    y_pred_test = best_ridge_model.predict(TEST_DATASETS[index])

    # Evaluar el rendimiento del modelo en los conjuntos de datos
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)

    # Almacenar los resultados para cada dataset
    results_ridge.append({
        "train_r2": train_r2,
        "train_mse": train_mse,
        "test_r2": test_r2,
        "test_mse": test_mse
    })

# Convertir los resultados en un DataFrame para mostrarlos
df_ridge = pd.DataFrame(results_ridge)

# Imprimir los resultados
print(df_ridge)

Procesando datasets Ridge:   0%|          | 0/6 [00:00<?, ?it/s]/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.89757e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=5.58068e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.54259e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.34376e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrit

Mejores hiperparámetros para el dataset 1: {'alpha': 100, 'max_iter': 100, 'tol': np.float64(0.0001)}


Procesando datasets Ridge:  33%|███▎      | 2/6 [02:37<05:04, 76.22s/it]

Mejores hiperparámetros para el dataset 2: {'alpha': 100, 'max_iter': 100, 'tol': np.float64(0.0001)}


Procesando datasets Ridge:  50%|█████     | 3/6 [03:36<03:25, 68.34s/it]

...

Mejores hiperparámetros para el dataset 4: {'alpha': 1, 'max_iter': 100, 'tol': np.float64(0.0001)}


Procesando datasets Ridge:  83%|████████▎ | 5/6 [05:34<01:02, 62.60s/it]

Mejores hiperparámetros para el dataset 5: {'alpha': 0.01, 'max_iter': 100, 'tol': np.float64(0.0001)}


Procesando datasets Ridge: 100%|██████████| 6/6 [06:33<00:00, 65.65s/it]

Mejores hiperparámetros para el dataset 6: {'alpha': 1, 'max_iter': 100, 'tol': np.float64(0.0001)}
   train_r2     train_mse   test_r2      test_mse
0  0.999693  3.468498e+06  0.998737  4.613747e+06
1  0.540903  5.185710e+09  0.477522  1.909152e+09
2  0.999693  3.467279e+06  0.998773  4.482677e+06
3  0.540735  5.187616e+09  0.487308  1.873394e+09
4  0.999413  6.633954e+06  0.997957  7.464642e+06
5  0.516088  5.466011e+09  0.531888  1.710497e+09
